In [ ]:
# EE219 - Project 5 Popularity Prediction on Twitter
# Author: Zhengshuang Ren
# Date: March 19, 2017

import pandas as pd
import numpy as np
import nltk
import json
import math
import string
import datetime, time
import matplotlib.pyplot as plt

import statsmodels.api as sm
from sklearn import linear_model

from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn import svm

import re
import nltk
from nltk import stem
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.tokenize import word_tokenize
from operator import attrgetter

from sklearn.decomposition import TruncatedSVD,PCA
from scipy.sparse.linalg import svds
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,roc_curve,accuracy_score,precision_score,recall_score
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import KFold,cross_val_predict
from scipy.sparse import csr_matrix
from sklearn.utils import shuffle


In [ ]:
# Problem 1
def problem_1(tag): # tag: 0-5
    tagnames = ["gohawks", "gopatriots", "nfl", "patriots", "sb49", "superbowl"]
    filepath = "tweet_data/tweets_#" + tagnames[tag] + ".txt"

    start_date = datetime.datetime(2015,1,18, 15,30,0) 
    end_date = datetime.datetime(2015,2,8, 15,30,0)
    mintime = int(time.mktime(start_date.timetuple())) 
    maxtime = int(time.mktime(end_date.timetuple())) 
    total_hours = math.ceil(float(maxtime-mintime)/3600) # 504 hours

    num_tweets = 0
    num_followers = 0
    num_retweets = 0
    num_followers_list = []
    num_tph = []
    users_id = []
    for line in open(filepath, 'r'):
        tweet = json.loads(line)
        tweet_time = tweet["firstpost_date"]
        if tweet_time >= mintime:
            if tweet_time >= maxtime:
                break;
        num_hour = int(tweet["firstpost_date"] - mintime) / 3600
        num_tweets += 1
        num_followers_list.append(tweet["author"]["followers"])
        num_retweets += tweet["metrics"]["citations"]["total"]
        users_id.append(tweet["tweet"]["user"]["id_str"])
        num_tph.append(num_hour)

    [unique_users_id, index] = np.unique(users_id, return_index = True) 
    for i in index:
        num_followers += num_followers_list[i]

    avg_num_tweet = float(num_tweets)/total_hours
    avg_num_followers_unique = float(num_followers)/len(unique_users_id) 
    avg_num_followers_duplic = sum(num_followers_list)/len(users_id)
    avg_num_retweets = float(num_retweets)/num_tweets
    
    print(avg_num_tweet)
    print(avg_num_followers_unique)
    print(avg_num_followers_duplic)
    print(avg_num_retweets)

    plt.figure(figsize = (9,6))
    plt.hist(num_tph, total_hours)
    plt.xlim(0,total_hours - 1)
    plt.xticks(np.arange(0, total_hours, 25.0))
    plt.show()

In [ ]:
# Problem 2
# Features: (1)number of tweets (2)total number of retweets (3)sum of the number of followers of the users posting the hashtag,
#           (4)maximum number of followers of the users posting the hashtag (5)time of the day
def problem_2(tag): # tag = 0 - 5
    tagnames = ["gohawks", "gopatriots", "nfl", "patriots", "sb49", "superbowl"]
    filepath = "tweet_data/tweets_#" + tagnames[tag] + ".txt"

    start_date = datetime.datetime(2015,1,18, 15,30,0) 
    end_date = datetime.datetime(2015,2,8, 15,30,0)
    mintime = int(time.mktime(start_date.timetuple())) 
    maxtime = int(time.mktime(end_date.timetuple())) 
    total_hours = math.ceil(float(maxtime-mintime)/3600) # 504 hours
    
    dataset = np.zeros((total_hours, 5)) 
    X = np.zeros((total_hours-1, 5)) 
    Y = np.zeros((total_hours-1, 1)) 
    time_offset = 15.5
    for line in open(filepath, 'r'):
        tweet= json.loads(line)
        tweet_time = tweet["firstpost_date"]    
        if tweet_time >= mintime:
            if tweet_time >= maxtime:
                break;
            num_hour = int((tweet_time-mintime)/3600)
            dataset[num_hour][0] += 1 # Feature(1)
            dataset[num_hour][1] += tweet["tweet"]["retweet_count"] # Feature(2)
            dataset[num_hour][2] += tweet["author"]["followers"] # Feature(3)
            dataset[num_hour][3] = max(dataset[num_hour][3], tweet["author"]["followers"]) # Feature(4)
            dataset[num_hour][4] = round(time_offset + float(tweet_time-mintime)/3600)%24 # Feature(5)

    for index in range(0,len(dataset)):
        if dataset[index][0] == 0:
            dataset[index][4]=round(time_offset+float(index))%24

    X = dataset[0:total_hours-1]
    Y = dataset[1:total_hours,0]

    result = sm.OLS(Y, X).fit()
    print(result.summary())

In [ ]:
# Problem 3
def problem_3(tag): # tag = 0 - 5
    tagnames = ["gohawks", "gopatriots", "nfl", "patriots", "sb49", "superbowl"]
    filepath = "tweet_data/tweets_#" + tagnames[tag] + ".txt"

    start_date = datetime.datetime(2015,1,18, 15,30,0) 
    end_date = datetime.datetime(2015,2,8, 15,30,0)
    mintime = int(time.mktime(start_date.timetuple())) 
    maxtime = int(time.mktime(end_date.timetuple())) 
    total_hours = math.ceil(float(maxtime-mintime)/3600) # 504 hours
    
    dataset = np.zeros((total_hours,9)) 
    X = np.zeros((total_hours-1,9)) 
    Y = np.zeros((total_hours-1,1)) 
    user_list = []

    threshold = 1000
    current = 0
    for line in open(filepath, 'r'):
        tweet = json.loads(line)
        tweet_time = tweet["firstpost_date"]    
        if tweet_time >= mintime:
            if tweet_time >= maxtime:
                break;
            num_hour = int((tweet_time-mintime)/3600)
            dataset[num_hour][0] += 1 # number of tweets
            dataset[num_hour][1] += tweet["tweet"]["retweet_count"] # numebr of retweets
            dataset[num_hour][2] += tweet["author"]["followers"] # number of followers
            dataset[num_hour][3] = max(dataset[num_hour][3], tweet["author"]["followers"])
            dataset[num_hour][4] = round(time_offset+float(tweet_time-mintime)/3600)%24 #time of the day
            dataset[num_hour][5] += int(tweet["author"]["followers"] >= threshold) # numbrer of celebrity
            if num_hour != current: 
                dataset[current][6] = len(np.unique(user_list)) #number of authers per hour 
                current = num_hour
                user_list = []
            user_list.append(tweet["tweet"]["user"]["id_str"])
            dataset[num_hour][7] += tweet["highlight"].count("@") # mention
            dataset[num_hour][8] += tweet["metrics"]["citations"]["total"] # citation

    for index in range(0,len(dataset)):
        if dataset[index][0]==0:
            dataset[index][4]=round(time_offset+float(index))%24

    X = dataset[0:total_hours-1]
    Y = dataset[1:total_hours,0]

    result = sm.OLS(Y, X).fit()
    print(result.summary())

In [ ]:
tweet["metrics"]["citations"]["total"]

In [ ]:
# Problem 4
def problem_4(tag, period): # tag: 0-5  period: before(1), during(2), after(3)
    tagnames = ["gohawks","gopatriots","nfl","patriots","sb49","superbowl"]
    filepath = "tweet_data/tweets_#" + tagnames[tag]+".txt"
    if period == 1:
        start_date = datetime.datetime(2015,1,18, 15,30,0)
        end_date = datetime.datetime(2015,2,1, 8,0,0) 
    elif period == 2:
        start_date = datetime.datetime(2015,2,1, 8,0,0) 
        end_date = datetime.datetime(2015,2,1, 20,0,0) 
    elif period == 3:
        start_date = datetime.datetime(2015,2,1, 20,0,0)
        end_date = datetime.datetime(2015,2,8, 15,30,0)
    time_offset = 15.5 
    mintime = int(time.mktime(start_date.timetuple()))
    maxtime = int(time.mktime(end_date.timetuple()))
    total_hours = int(math.ceil(float(maxtime-mintime)/3600))

    dataset = np.zeros((total_hours,9))
    X= np.zeros((total_hours-1,9)) 
    Y = np.zeros((total_hours-1,1))
    user_list = []
    num_tph = []

    threshold = 1000 # celebrity threshold
    current = 0
    for line in open(filepath, 'r'):
        tweet=json.loads(line)
        tweet_time = tweet["firstpost_date"]    
        if tweet_time >= mintime:
            if tweet_time >= maxtime:
                break;
            num_hour = int((tweet_time - mintime)/3600)
            dataset[num_hour][0] += 1 
            dataset[num_hour][1] += tweet["tweet"]["retweet_count"] 
            dataset[num_hour][8] += tweet["metrics"]["citations"]["total"]
            dataset[num_hour][2] += tweet["author"]["followers"] 
            dataset[num_hour][3] = max(dataset[num_hour][3], tweet["author"]["followers"])
            dataset[num_hour][4] = round(time_offset+float(tweet_time-mintime)/3600)%24 
            dataset[num_hour][5] += int(tweet["author"]["followers"] >= threshold)
            if num_hour != current: 
                dataset[current][6] = len(np.unique(user_list))
                current = num_hour
                user_list = []
            user_list.append(tweet["tweet"]["user"]["id_str"])
            dataset[num_hour][7] += tweet["highlight"].count("@") 
            num_tph.append(num_hour)

    for index in range(0,len(dataset)):
        if dataset[index][0] == 0:
            dataset[index][4] = round(time_offset+float(index))%24

    X = dataset[0:total_hours-1]
    Y = dataset[1:total_hours,0]

    # 10-fold cross validation
    alpha = 0.01
    f10 = KFold(len(X), n_folds=10, shuffle=True, random_state=None)
    lr = linear_model.LinearRegression()
    rfr = RandomForestRegressor(n_estimators = 20,criterion = 'mse',max_depth = 4, max_features='auto',random_state=None)
    model = Pipeline([('poly', PolynomialFeatures(degree=2)),('linear',linear_model.LinearRegression(fit_intercept=True,normalize=True))])
    lasso = linear_model.Lasso(alpha,fit_intercept=True,normalize=True)
    ridge = linear_model.Ridge(alpha,fit_intercept=True,normalize=True)
    svr = svm.SVR()

    lr_err_list = []
    rfr_err_list = []
    poly_err_list = []
    lasso_err_list = []
    ridge_err_list = []
    svr_err_list = []
    count = 0
    for train_index, test_index in f10:
        X_train, X_test = X[train_index,:], X[test_index,:]
        y_train, y_test = Y[train_index], Y[test_index]

        lr = lr.fit(X_train,y_train) 
        rfr = rfr.fit(X_train,y_train) 
        poly = model.fit(X_train, y_train) 
        lasso = lasso.fit(X_train,y_train)
        ridge = ridge.fit(X_train,y_train)
        svr = svr.fit(X_train,y_train) 
        '''
        print (str(count) + ' Fold:')
        print ('Linear Regression: ' + str(lr.score(X_test,y_test)))
        print ('Random Forest Regression: ' + str(rfr.score(X_test,y_test)))
        print ('Polynomial Regression: ' + str(model.score(X_test,y_test)))
        print ('Lasso Regression: ' + str(lasso.score(X_test,y_test)))
        print ('Ridge Regression: ' + str(ridge.score(X_test,y_test)))
        print ('SVM Regression: ' + str(svr.score(X_test,y_test)))
        '''
        lr_err = np.mean(np.abs(lr.predict(X_test) - y_test))
        rfr_err = np.mean(np.abs(rfr.predict(X_test) - y_test))
        poly_err = np.mean(np.abs(poly.predict(X_test) - y_test))
        lasso_err = np.mean(np.abs(lasso.predict(X_test) - y_test))
        ridge_err = np.mean(np.abs(ridge.predict(X_test) - y_test))  
        svr_err = np.mean(np.abs(svr.predict(X_test) - y_test))

        lr_err_list.append(lr_err)
        rfr_err_list.append(rfr_err)
        poly_err_list.append(poly_err) 
        lasso_err_list.append(lasso_err)
        ridge_err_list.append(ridge_err) 
        svr_err_list.append(svr_err) 
        count += 1

    avg_err_lr=np.mean(lr_err_list)
    avg_err_rfr=np.mean(rfr_err_list)
    avg_err_poly=np.mean(poly_err_list)
    avg_err_lasso=np.mean(lasso_err_list)
    avg_err_ridge=np.mean(ridge_err_list)
    avg_err_svr=np.mean(svr_err_list)

    print ('Linear Regression: '+ str(avg_err_lr))
    print ('Random Forest Regression: '+ str(avg_err_rfr))
    print ('Polynomial Regression: '+ str(avg_err_poly))
    print ('Lasso Regression: '+ str(avg_err_lasso))
    print ('Ridge Regression: '+ str(avg_err_ridge))
    print ('SVM Regression: '+ str(avg_err_svr))

In [ ]:
# Problem 5
def findDominatingTag(file_num): # 0-9
    filename =["sample1_period1.txt","sample2_period2.txt","sample3_period3.txt","sample4_period1.txt","sample5_period1.txt"
                ,"sample6_period2.txt","sample7_period3.txt","sample8_period1.txt","sample9_period2.txt","sample10_period3.txt"]
    filepath = 'test_data/' + filename[file_num]
    tagnames = ["gohawks","gopatriots","nfl","patriots","sb49","superbowl"]

    tweets = []
    highlight = []
    tags = []
    tags_count = np.zeros(6)
    max_tag = 0
    for line in open(filepath, 'r'):
        tweet = json.loads(line)
        tweets.append(tweet)
        highlight.append(tweet["highlight"])    
    for line in highlight:
        for tagname in tagnames:
            if tagname in line.lower():
                tags.append(tagname)
    for index in range(0,6):
        for tag in tags:
            if tag == tagnames[index]:
                tags_count[index] += 1;
    for index in range(0,6):
        if(tags_count[index] == np.max(tags_count)):
            max_tag_index = index;
    return tagnames[max_tag_index]

In [ ]:
# Find the dominating tag in each file
print(findDominatingTag(0))
print(findDominatingTag(1))
print(findDominatingTag(2))
print(findDominatingTag(3))
print(findDominatingTag(4))
print(findDominatingTag(5))
print(findDominatingTag(6))
print(findDominatingTag(7))
print(findDominatingTag(8))
print(findDominatingTag(9))

In [ ]:
# Collect Data
def collect_test_data(file_num): # file_num: 0-9
    filename =["sample1_period1.txt","sample2_period2.txt","sample3_period3.txt","sample4_period1.txt","sample5_period1.txt"
                ,"sample6_period2.txt","sample7_period3.txt","sample8_period1.txt","sample9_period2.txt","sample10_period3.txt"]
    filepath = 'test_data/' + filename[file_num]
    #period = int(((filename[file_num].split('.')[0]).split('_')[1])[6])
    time_offset = 15.5 
    total_hours = 6
    mintime = math.inf
    maxtime = -math.inf
    dataset = np.zeros((6, 9)) 
    user_list = []
    threshold = 1000
    current = 0
    num_tph = []
    num_hour = 0
    tweets = []
    for line in open(filepath, 'r'):
        tweet = json.loads(line)
        mintime = min(tweet["firstpost_date"], mintime)
        maxtime = max(tweet["firstpost_date"], maxtime)
  
    for line in open(filepath, 'r'):
        tweet = json.loads(line)
        tweet_time = tweet["firstpost_date"]    
        num_hour = int((tweet_time - mintime)/3600)
        dataset[num_hour][0] += 1 
        dataset[num_hour][1] += tweet["tweet"]["retweet_count"] 
        dataset[num_hour][2] += tweet["author"]["followers"] 
        dataset[num_hour][3] = max(dataset[num_hour][3], tweet["author"]["followers"])
        dataset[num_hour][4] = round(time_offset+float(tweet_time-mintime)/3600)%24 
        dataset[num_hour][5] += int(tweet["author"]["followers"] >= threshold)
        if num_hour != current: 
            dataset[current][6] = len(np.unique(user_list))
            current = num_hour
            user_list = []
        user_list.append(tweet["tweet"]["user"]["id_str"])
        dataset[num_hour][7] += tweet["highlight"].count("@") 
        dataset[num_hour][8] += tweet["metrics"]["citations"]["total"]
        num_tph.append(num_hour)
    for index in range(0,len(dataset)):
        if dataset[index][0] == 0:
            dataset[index][4] = round(time_offset+float(index))%24 
            
    return dataset

In [ ]:
dataset_test_0 = collect_test_data(0)
dataset_test_1 = collect_test_data(1)
dataset_test_2 = collect_test_data(2)
dataset_test_3 = collect_test_data(3)
dataset_test_4 = collect_test_data(4)
dataset_test_5 = collect_test_data(5)
dataset_test_6 = collect_test_data(6)
dataset_test_7 = collect_test_data(7)
dataset_test_8 = collect_test_data(8)
dataset_test_9 = collect_test_data(9)

In [ ]:
# Generate Regression Model
def generate_reg_model(tagname, period): # tag: 0-5  period: before(1), during(2), after(3)
    filepath = "tweet_data/tweets_#" + tagname + ".txt"
    if period == 1:
        start_date = datetime.datetime(2015,1,18, 15,30,0)
        end_date = datetime.datetime(2015,2,1, 8,0,0) 
    elif period == 2:
        start_date = datetime.datetime(2015,2,1, 8,0,0) 
        end_date = datetime.datetime(2015,2,1, 20,0,0) 
    elif period == 3:
        start_date = datetime.datetime(2015,2,1, 20,0,0)
        end_date = datetime.datetime(2015,2,8, 15,30,0)
    time_offset = 15.5 
    mintime = int(time.mktime(start_date.timetuple()))
    maxtime = int(time.mktime(end_date.timetuple()))
    total_hours = int(math.ceil(float(maxtime-mintime)/3600))

    dataset = np.zeros((total_hours,9))
    X= np.zeros((total_hours-1,9)) 
    Y = np.zeros((total_hours-1,1))
    user_list = []
    num_tph = []

    threshold = 1000 # celebrity threshold
    current = 0
    for line in open(filepath, 'r'):
        tweet=json.loads(line)
        tweet_time = tweet["firstpost_date"]    
        if tweet_time >= mintime:
            if tweet_time >= maxtime:
                break;
            num_hour = int((tweet_time - mintime)/3600)
            dataset[num_hour][0] += 1 
            dataset[num_hour][1] += tweet["tweet"]["retweet_count"] 
            dataset[num_hour][8] += tweet["metrics"]["citations"]["total"]
            dataset[num_hour][2] += tweet["author"]["followers"] 
            dataset[num_hour][3] = max(dataset[num_hour][3], tweet["author"]["followers"])
            dataset[num_hour][4] = round(time_offset+float(tweet_time-mintime)/3600)%24 
            dataset[num_hour][5] += int(tweet["author"]["followers"] >= threshold)
            if num_hour != current: 
                dataset[current][6] = len(np.unique(user_list))
                current = num_hour
                user_list = []
            user_list.append(tweet["tweet"]["user"]["id_str"])
            dataset[num_hour][7] += tweet["highlight"].count("@") 
            num_tph.append(num_hour)

    for index in range(0,len(dataset)):
        if dataset[index][0] == 0:
            dataset[index][4] = round(time_offset+float(index))%24

    X = dataset[0:total_hours-1]
    Y = dataset[1:total_hours,0]
    
    rfr = RandomForestRegressor(n_estimators = 20,criterion = 'mse',max_depth = 4, max_features='auto',random_state=None) 
    rfr = rfr.fit(X, Y) 
    ols = sm.OLS(Y, X).fit()
    
    return rfr, ols

In [ ]:
# Problem 5 main
def problem_5(file_num): # file_num: 0-9 model: 'rfr', 'ols'
    filename =["sample1_period1.txt","sample2_period2.txt","sample3_period3.txt","sample4_period1.txt","sample5_period1.txt"
                ,"sample6_period2.txt","sample7_period3.txt","sample8_period1.txt","sample9_period2.txt","sample10_period3.txt"]
    filepath = 'test_data/' + filename[file_num]
    period = int(((filename[file_num].split('.')[0]).split('_')[1])[6])
    
    tag = findDominatingTag(file_num)
    test_data = collect_test_data(file_num)
    
    rfr, ols = generate_reg_model(tag, period)
       
    rfr_predict = rfr.predict(test_data)
    ols_predict = ols.predict(test_data)

    print(rfr_predict)
    print(ols_predict)

In [ ]:
# Problem 6
def parsetweet(filename):
	tweet_text=[]
	tweet_location=[]
	with open(filename) as tweet_data:
		for line in tweet_data:
			individual_tweet=json.loads(line)
			tweet_text.append(individual_tweet["tweet"]["text"])
			try:
				tweet_location.append(individual_tweet["tweet"]["user"]["description"].encode('utf-8')+individual_tweet["tweet"]["user"]["location"].encode('utf-8'))
			except AttributeError:
				tweet_location.append("unknown")
	print (len(tweet_text),len(tweet_location))
	select_tweet_text=[]
	select_tweet_location=[]
	
	for i in range(len(tweet_location)):
		if ("WA" in tweet_location[i]) or ("Washington" in tweet_location[i]) or ("Seattle" in tweet_location[i]):
			select_tweet_text.append(tweet_text[i])
			select_tweet_location+=[0]
		elif ("MA" in tweet_location[i]) or ("Massachusetts" in tweet_location[i]) or ("Boston" in tweet_location[i]):
			select_tweet_text.append(tweet_text[i])
			select_tweet_location+=[1]
	return select_tweet_text,select_tweet_location

In [ ]:
def problem_6():
    tweet_superbowl="tweet_data/tweets_#superbowl.txt"
    tweet_text,tweet_location=parsetweet(tweet_superbowl)

    count_vect = CountVectorizer(token_pattern='[A-Za-z0-9]+',stop_words='english')
    tweet_count = count_vect.fit_transform(tweet_text)

    tfidf_transformer=TfidfTransformer()
    tweet_tfidf=tfidf_transformer.fit_transform(tweet_count)
    print (tweet_tfidf.shape)
    print (len(tweet_location))
    #tweet_tfidf=csr_matrix(tweet_tfidf,dtype=np.float32)
    svd=TruncatedSVD(n_components=5)
    X_tweet=svd.fit_transform(tweet_tfidf)
    #X_tweet,s,vt=svds(tweet_tfidf,k=5)
    tweet_location=np.asarray(tweet_location)
    X_tweet,tweet_location=shuffle(X_tweet,tweet_location)

    #soft SVM

    accuracy=[0]*5
    precision=[0]*5
    recall=[0]*5
    for i in range(0,5):
        soft_svc=SVC(C=10**(i-2),kernel='linear',probability=True)
        #Y_predict=cross_val_predict(soft_svc,X_tweet,tweet_location,cv=3)
        Y_predict_proba=cross_val_predict(soft_svc,X_tweet,tweet_location,cv=3,method="predict_proba")
        Y_predict=Y_predict_proba[:,1]>Y_predict_proba[:,0]
        Y_predict=[int(x) for x in Y_predict]
        accuracy[i]=accuracy_score(Y_predict,tweet_location)
        precision[i]=precision_score(Y_predict,tweet_location)
        recall[i]=recall_score(Y_predict,tweet_location)
        print ("*****************************************************************************************")
        print ("The value of Gamma is %f" % 10**(i-2))
        print (accuracy[i])
        print (precision[i])
        print (recall[i])
        print (confusion_matrix(Y_predict,tweet_location))

        fpr, tpr, thresholds = roc_curve(tweet_location, Y_predict_proba[:,1])
        s = 'Soft SVM ROC for Gamma=%f' % 10**(i-2)

        plt.plot(fpr, tpr)                                    
        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(s)

        plt.savefig('gamma='+str(10**(i-2))+'.png')
        plt.show()

    nbCLF=GaussianNB()
    Y_predict_proba=cross_val_predict(nbCLF,X_tweet,tweet_location,cv=3,method="predict_proba")
    #Y_predict=cross_val_predict(nbCLF,X_tweet,tweet_location,cv=3)
    Y_predict=Y_predict_proba[:,1]>Y_predict_proba[:,0]
    Y_predict=[int(x) for x in Y_predict]
    nb_accuracy=accuracy_score(Y_predict,tweet_location)
    nb_precision=precision_score(Y_predict,tweet_location)
    nb_recall=recall_score(Y_predict,tweet_location)
    print ("*****************************************************************************************")
    print ("Naive Bayes Classifier")
    print (nb_accuracy)
    print (nb_precision)
    print (nb_recall)
    print (confusion_matrix(Y_predict,tweet_location))

    fpr, tpr, thresholds = roc_curve(tweet_location, Y_predict_proba[:,1])
    s = 'Naive Bayes Classifier'
    plt.plot(fpr, tpr)                                    
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(s)

    plt.savefig('Gaussian.png')
    plt.show()

In [ ]:
# Problem 7
emoticons_str = r"""(?:
        [:=;] # Eyes
        [oO\-]?# Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""

emoticons_str_pos = r"""(?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\]/\\OpP] # Mouth
    )"""
emoticons_str_neg = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [\(\[] # Mouth
    )"""
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
emoticon_pos_re = re.compile(r'^'+emoticons_str_pos+'$', re.VERBOSE | re.IGNORECASE)
emoticon_neg_re = re.compile(r'^'+emoticons_str_neg+'$', re.VERBOSE | re.IGNORECASE)

def tokenize(s):
    return tokens_re.findall(s)
    
def remove_punctuation(text):
    return re.sub(r"\p{P}+", "", text)
try:
    # Wide UCS-4 build
    emoji = re.compile(u'['
        u'\U0001F300-\U0001F64F'
        u'\U0001F680-\U0001F6FF'
        u'\u2600-\u26FF\u2700-\u27BF]+', 
        re.UNICODE)
except re.error:
    # Narrow UCS-2 build
    emoji = re.compile(u'('
        u'\ud83c[\udf00-\udfff]|'
        u'\ud83d[\udc00-\ude4f\ude80-\udeff]|'
        u'[\u2600-\u26FF\u2700-\u27BF])+', 
        re.UNICODE)
    
def replaceTwoOrMore(sentence):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", sentence)

def preprocess(tweet, lowercase=False):
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',tweet) # delete url
    tweet = re.sub('@[^\s]+',' ',tweet) # delet @username
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # replace #hashtag with hash tag   
    tweet = re.sub(u"(\u2018|\u2019)", "'", tweet) # map \u2018 and \u2019 into '
    tweet = replaceTwoOrMore(tweet)
    tweet = emoji.sub('',tweet)
    tokens = tokenize(tweet)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    tokens = [w if emoticon_re.search(w) else w.strip('\'"?,.!&;:/') for w in tokens]
    tokens = [emoticon_to_word(w) for w in tokens]
    tokens = [w for w in tokens if not w in nltk.corpus.stopwords.words('english')] 
    if u'' in tokens: tokens.remove('')
    return tokens

def emoticon_to_word(word):
    if emoticon_re.search(word):
        if emoticon_neg_re.search(word):
            word = u'sad'
        else:
            word = u'happy'
    return word

In [ ]:
def problem_7(tag, period):
    tagname=["gohawks","gopatriots","nfl","patriots","sb49","superbowl"]
    filepath="tweet_data/tweets_#"+tagname[tag]+".txt"
    
    if period == 1:
        start_date = datetime.datetime(2015,1,18, 15,30,0) 
        end_date = datetime.datetime(2015,2,1, 8,0,0) 
    elif period == 2:
        start_date = datetime.datetime(2015,2,1, 8,0,0)
        end_date = datetime.datetime(2015,2,1, 20,0,0) 
    elif period == 3:
        start_date = datetime.datetime(2015,2,1, 20,0,0) 
        end_date = datetime.datetime(2015,2,8, 15,30,0)
    elif period == 4:
        start_date = datetime.datetime(2015,1,18, 15,30,0) 
        end_date = datetime.datetime(2015,2,8, 15,30,0)

    time_offset = 15.5
    mintime = int(time.mktime(start_date.timetuple())) 
    maxtime = int(time.mktime(end_date.timetuple()))


    stopWords = []
    total_hours = int(math.ceil(float(maxtime-mintime)/3600))
    pos_num = np.zeros(total_hours)
    neg_num = np.zeros(total_hours)
    pos_excitment = np.zeros(total_hours)
    neg_excitment = np.zeros(total_hours)
    tweet_num = np.zeros(total_hours)
    tweet_polarity = []
    current = 0
    for line in open(filepath, 'r'):
        tweet = json.loads(line)
        tweet_time = tweet["firstpost_date"]  
        if tweet_time >= mintime:
            if tweet_time >= maxtime:
                break;
            num_hour = int((tweet_time-mintime)/3600)
            tweet_num[num_hour] += 1
            text = tweet["tweet"]["text"]
            text = preprocess(text, lowercase = True)
            text_pos_score = []
            text_neg_score = []
            for word in text:
                senti_word = list(swn.senti_synsets(word))            
                if len(senti_word) > 0:
                    word_pos_score = np.mean([senti_word[i].pos_score() for i in range(len(senti_word))])
                    word_neg_score = np.mean([senti_word[j].neg_score() for j in range(len(senti_word))])
                    text_pos_score.append(word_pos_score)
                    text_neg_score.append(word_neg_score)
            if len(senti_word) > 0:
                if np.mean(text_pos_score) > np.mean(text_neg_score):
                    pos_num[(num_hour)] += 1
                    pos_excitment[(num_hour)] += np.sum(text_pos_score)
                    tweet_polarity.append(1)
                elif np.mean(text_pos_score) < np.mean(text_neg_score):
                    neg_num[num_hour] += 1
                    neg_excitment[(num_hour)] += np.sum(text_neg_score)
                    tweet_polarity.append(-1)
                else:
                    tweet_polarity.append(0)

    total_pos_num = sum(pos_num)
    total_neg_num = sum(neg_num)
    ttn = sum(tweet_num)
    print(ttn)
    print(total_pos_num)
    print(total_neg_num)
    
    plt.figure(figsize = (9,6))
    plt.plot(range(total_hours),pos_num,'o-', color = 'red')
    plt.plot(range(total_hours),neg_num,'o-', color = 'blue')
    plt.legend(['Postive','Negative'])
    plt.xlim(0,total_hours - 1)
    plt.show()
